In [6]:
import rasterio
from rasterio import rio
from rasterio.mask import mask
import geopandas as gpd
from shapely import Polygon, Point
from shapely.geometry import mapping
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import sys
import glob

### Loading the NTL data

In [2]:
# rasterio_img = rasterio.open("./data/earth_observation_group/annual/2012/VNL_v2_npp_201204-201212_global_vcmcfg_c202101211500.average.tif")

## Getting radiance value inside a polygon defined by geocoordinates

In [3]:
def check_polar_limit(geom_coords, north_latitude_limit, south_latitude_limit):
    for coord in geom_coords:
        if coord[1] > north_latitude_limit or coord[1] < south_latitude_limit:
            return True
    return False
    
def get_polar_regions(north_latitude_limit, south_latitude_limit, country_geometry_dict):
    polar_regions = []
    
    for country, country_geometry in country_geometry_dict.items():
        if country_geometry.geom_type == "MultiPolygon":
            for i in range(len(country_geometry.geoms)):
                geom_coords = country_geometry.geoms[i].exterior.coords[::-1]
                if check_polar_limit(geom_coords, north_latitude_limit, south_latitude_limit):
                    polar_regions.append(country)
                    break
                
        else:
            geom_coords = country_geometry.exterior.coords[::-1]
            if check_polar_limit(geom_coords, north_latitude_limit, south_latitude_limit):
                polar_regions.append(country)
                break
                
    return polar_regions
    

def get_masked_polygons(rasterio_img, country_geometry_dict):
    country_mappings = {country: mapping(country_geometry) for country, country_geometry in country_geometry_dict.items()}
    country_masked_polygons = dict()
    for country, country_geometry in tqdm(country_mappings.items()):
        out_image, _ = mask(rasterio_img, [country_geometry], crop=True)
        country_masked_polygons[country] = out_image
    
    return country_masked_polygons


## Getting radiance value inside a polygon defined by geocoordinates
def get_polygon_radiance_value(masked_region_image, polygon, no_rasterio_data):
    try:
#         # transform to GeJSON format
#         geoms = [mapping(polygon)]

#         # extract the raster values values within the polygon
#         out_image, out_transform = mask(rasterio_img, geoms, crop=True)

#         # no data values of the original raster
#         no_data=rasterio_img.nodata

#         del rasterio_img

        # extract the values of the masked array
#         data = np.squeeze()
        data = np.reshape(masked_region_image.data.tolist(), (masked_region_image.shape[1], masked_region_image.shape[2]))

        # extract the row, columns of the valid values
        row, col = np.where(data != no_rasterio_data) 
        elev = np.extract(data != no_rasterio_data, data)
        rad_sum = round(np.sum(elev), 2)
    except Exception as error:
        print("Error: {} while evaluating radiance sum.".format(error))
    return rad_sum

### Radiance value sum for a country

In [5]:
rasterio_img = rasterio.open("./data/earth_observation_group/annual/1992/F101992.v4b.global.intercal.stable_lights.avg_vis.tif")

In [4]:
all_country_shapefile = gpd.read_file("./data/country_geometry/all_country/ne_10m_admin_0_countries.shp")

# extract the geometry in GeoJSON format
all_country_geometry = all_country_shapefile.geometry.values # list of shapely geometries
country_names = list(all_country_shapefile.SOV_A3)
country_geometry_dict = {country_names[i]: all_country_geometry[i] for i in range(len(all_country_geometry))}

In [7]:
country_name = "TUR"
country_geometry = country_geometry_dict[country_name]

In [6]:
# for i in range(len(country_geometry.geoms)):
#     print(check_polar_limit(country_geometry.geoms[i].exterior.coords[::-1], 75.0, -65.0))

In [7]:
# tmp = get_polygon_radiance_value(rasterio_img, country_geometry)
# all_countries_rad_sum_dict = dict()

# for country, country_geometry in tqdm(country_geometry_dict.items()):
#     radiance_sum = get_polygon_radiance_value(rasterio_img, country_geometry)
#     all_countries_rad_sum_dict[country] = radiance_sum

In [8]:
# for key in ["RUS", "CAN", "NOR", "ATA"]:
#     del country_geometry_dict[key]
# tmp = {country: mapping(country_geometry) for country, country_geometry in country_geometry_dict.items()}

In [5]:
print("countries before deleting polar regions: ", len(country_geometry_dict))
polar_regions = get_polar_regions(75.0, -65.0, country_geometry_dict)
polar_regions.append("KIR")
for key in polar_regions:
    del country_geometry_dict[key]
    
print("countries after deleting polar regions: ", len(country_geometry_dict))

countries before deleting polar regions:  209
countries after deleting polar regions:  204


In [22]:
base_path = "./data/earth_observation_group/annual/"
for file in glob.glob("{}*".format(base_path)):
    year = file.split("/")[-1]
    if int(year) in np.arange(1992, 2012+1):
        for rasterio_file in glob.glob(base_path+year+"/*"):
            if rasterio_file.split("/")[-1].split(".")[-1] == "tif":
                rasterio_img = rasterio.open(rasterio_file)
                masked_polygons = get_masked_polygons(rasterio_img, country_geometry_dict)
                g_min, g_max = 10**5, -10**5

                for country in masked_polygons:
                    g_min = min(g_min, np.min(masked_polygons[country]))
                    g_max = max(g_max, np.max(masked_polygons[country]))

                print(year, g_min, g_max)

In [26]:
base_path = "./data/earth_observation_group/annual/"
for file in glob.glob("{}*".format(base_path)):
    year = file.split("/")[-1]
    if int(year) in np.arange(1992, 2012+1):
        for rasterio_file in glob.glob(base_path+year+"/*"):
            if rasterio_file.split("/")[-1].split(".")[-1] == "tif":
                print(rasterio_file.split("/")[-1].split(".")[0][:3])
            

F15
F14
F15
F16
F16
F15
F16
F15
F15
F14
F14
F15
F14
F15
F18
F12
F14
F18
F14
F12
F18
F12
F14
F10
F10
F12
F12
F12
F15
F16
F16
F16
F10


In [10]:
# np.sum(masked_polygons["TUR"])

1784666.9

In [14]:
np.max(masked_polygons["TUR"]), np.min(masked_polygons["TUR"])

(63.0, 0.0)

In [27]:
# # dir(country_geometry)
# # north_circle_coords = []
# north_circle_coords = [(long, 75) for long in range(-180, 181)]
# north_circle_polygon = Polygon(north_circle_coords)

In [45]:
# rasterio_img.statistics(1)
# round(np.sum(tmp), 2)
# country_geometry.intersects(north_circle_polygon)
# north_circle_coords

# for coords in north_circle_coords:
#     print(country_geometry.contains(Point(coords)))
# tmp_point = Point((10.7522, 75.9139))
# country_geometry.contains(tmp_point)

# country_geometry.intersects(north_circle_polygon)

True

In [3]:
df = pd.read_csv("./data/earth_observation_group/annual/2014/countries_ntl_sum.csv")

In [4]:
df

,Region Code,annual_ntl_sum
0,IDN,1335862.40
1,MYS,1126525.60
2,CHL,1350989.60
3,BOL,400901.60
4,PER,550756.94
...,...,...
199,BHR,180006.28
200,PGA,-0.19
201,BJN,0.00
202,SER,-0.01


### Debugging

In [39]:
regions_selected = set(pd.read_csv("regions_selected.csv", header=None)[0].values)
regions_evaluated = set(pd.read_csv("regions_evaluated.csv", header=None)[0].values)

missing_regions = regions_selected - regions_evaluated

In [ ]:
missing_regions_rad = dict()

for country_name in tqdm(list(missing_regions)):
    country_geometry = country_geometry_dict[country_name]
    missing_regions_rad[country_name] = get_polygon_radiance_value(rasterio_img, country_geometry)

 20%|████████████▋                                                  | 22/109 [01:07<02:37,  1.81s/it]

### Radiance value sum for a city

In [56]:
all_cities_shapefile = gpd.read_file("./data/country_geometry/netherlands/NLD_adm2.shp")

city_geometry_dict = {}
for city_name, polygon in zip(all_cities_shapefile["NAME_2"], all_cities_shapefile["geometry"]):
    if city_name is not None:
        city_geometry_dict[city_name.lower()] = polygon

In [62]:
city_name = "amsterdam"
city_geometry = city_geometry_dict[city_name]

get_polygon_radiance_value(rasterio_img, city_geometry)

28709.29

## GDP 

In [81]:
gdp_df = pd.read_csv("./data/gdp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4901850.csv")
gdp_df["Country Name"] = gdp_df["Country Name"].str.lower()

In [95]:
## Returns GDP of a country in USD billions
def get_gdp(df, country_name, year):
    gdp = list(df[df["Country Name"] == country_name][str(year)])
    assert len(gdp) > 0, "Country {} does not exist in the GDP database".format(country_name)
    assert gdp is not None, "GDP value not found for {} for the year {}".format(country_name, year) 
    return round(gdp[0]//10**9, 2)

In [100]:
get_gdp(gdp_df, "netherlands", 2021)

1012.0

## GDP disaggregation

In [118]:
country_name = "netherlands"
city_name = "leiden"
year = 2021

In [119]:
country_geometry = country_geometry_dict[country_name]
city_geometry = city_geometry_dict[city_name]

In [ ]:
country_radiance_sum = get_polygon_radiance_value(rasterio_img, country_geometry)

In [136]:
country_gdp = get_gdp(gdp_df, country_name, year)

city_radiance_dict = {}
city_gdp_dict = {}
for city_name, city_geometry in city_geometry_dict.items():
    city_radiance = get_polygon_radiance_value(rasterio_img, city_geometry)
    city_radiance_dict[city_name] = city_radiance
    
    city_gdp_proportion = city_radiance/country_radiance_sum
    
    city_gdp = city_gdp_proportion * country_gdp
    city_gdp_dict[city_name] = (city_gdp, round(city_gdp_proportion, 6))    

In [137]:
sorted(city_radiance_dict.items(), key= lambda x: x[1], reverse=True)

[('noordoostpolder', 81878.71),
 ('naaldwijk', 72972.73),
 ('maasland', 72839.21),
 ('rotterdam', 68345.0),
 ('de lier', 55691.46),
 ('schipluiden', 53373.51),
 ('zevenhuizen-moerkapelle', 53181.71),
 ('wieringermeer', 52560.44),
 ('waddinxveen', 48920.21),
 ("'s-gravenzande", 35864.68),
 ('pijnacker-nootdorp', 35547.84),
 ('berkel en rodenrijs', 33957.75),
 ('reimerswaal', 31409.28),
 ('haarlemmermeer', 30077.03),
 ('emmen', 29878.89),
 ('amsterdam', 28709.29),
 ('monster', 28526.59),
 ('bleiswijk', 27096.37),
 ('maasbree', 20705.75),
 ('steenbergen', 20192.33),
 ('westvoorne', 18229.07),
 ("'s-gravenhage", 17523.45),
 ('wateringen', 17510.57),
 ('bergschenhoek', 14434.07),
 ('eindhoven', 13686.76),
 ('terneuzen', 11349.78),
 ('zaltbommel', 11167.55),
 ('utrecht', 11114.29),
 ('moerdijk', 9738.2),
 ('almere', 9285.37),
 ('venlo', 8676.56),
 ('apeldoorn', 8630.11),
 ('aalsmeer', 8131.75),
 ('tilburg', 7948.19),
 ('breda', 7652.17),
 ('someren', 7361.21),
 ('zoetermeer', 7139.79),
 ('gr

In [138]:
sorted(city_gdp_dict.items(), key= lambda x: x[1], reverse=True)

[('noordoostpolder', (44.58822633368545, 0.04406)),
 ('naaldwijk', (39.738347140873586, 0.039267)),
 ('maasland', (39.66563690911648, 0.039195)),
 ('rotterdam', (37.21825037028223, 0.036777)),
 ('de lier', (30.327583609138316, 0.029968)),
 ('schipluiden', (29.06531067848069, 0.028721)),
 ('zevenhuizen-moerkapelle', (28.960863236516822, 0.028617)),
 ('wieringermeer', (28.622541743978303, 0.028283)),
 ('waddinxveen', (26.64020226712685, 0.026324)),
 ("'s-gravenzande", (19.530626083693814, 0.019299)),
 ('pijnacker-nootdorp', (19.358086315644645, 0.019129)),
 ('berkel en rodenrijs', (18.492179991388564, 0.018273)),
 ('reimerswaal', (17.10437408720899, 0.016902)),
 ('haarlemmermeer', (16.378878234464697, 0.016185)),
 ('emmen', (16.270978254533937, 0.016078)),
 ('amsterdam', (15.634055793006658, 0.015449)),
 ('monster', (15.534563886610425, 0.01535)),
 ('bleiswijk', (14.755717064683655, 0.014581)),
 ('maasbree', (11.275613250486083, 0.011142)),
 ('steenbergen', (10.996023022889181, 0.010866)

In [73]:
209-14

67